# NOTE for TransOMCS

Author: Doublu

Date: 2020年10月29日20:22:50



如果想要直接使用TransOMCS，作者提供了资源下载[TransOMCS](https://hkustconnect-my.sharepoint.com/:u:/g/personal/hzhangal_connect_ust_hk/EVeNd_qvealEiTi7gs0Xu6sBbPIZI5ncD7Z1MBMdOz5CXw?e=VWCAbg)

查看前几行

In [9]:
with open('./Data/TransOMCS_full.txt/TransOMCS_full.txt', 'r') as f:
    for i in range(10):
        print(f.readline(), end="")

student	AtLocation	school	1.0
building	AtLocation	city	1.0
sugar	AtLocation	coffee	1.0
government	AtLocation	city	1.0
school	AtLocation	city	1.0
sign	AtLocation	book	1.0
title	AtLocation	book	1.0
curator	AtLocation	museum	1.0
leader	AtLocation	group	1.0
population	AtLocation	city	1.0


In [1]:
import pandas as pd
result = pd.read_csv('./Data/TransOMCS_full.txt/TransOMCS_full.txt', header=None, dtype=str, encoding='utf-8', sep='\t')
result.columns = ['head', 'relation', 'tail', 'score']

In [2]:
result

,head,relation,tail,score
0,student,AtLocation,school,1.0
1,building,AtLocation,city,1.0
2,sugar,AtLocation,coffee,1.0
3,government,AtLocation,city,1.0
4,school,AtLocation,city,1.0
...,...,...,...,...
18481602,nein,ReceivesAction,nein,0.00
18481603,concurrence,ReceivesAction,concur,0.00
18481604,slither,ReceivesAction,slither,0.00
18481605,establishment,ReceivesAction,ban,0.00


In [3]:
result.shape

(18481607, 4)

In [4]:
result['relation'].unique()

array(['AtLocation', 'CapableOf', 'Causes', 'CausesDesire', 'CreatedBy',
       'DefinedAs', 'Desires', 'HasA', 'HasPrerequisite', 'HasProperty',
       'HasSubevent', 'HasFirstSubevent', 'HasLastSubevent', 'InstanceOf',
       'MadeOf', 'MotivatedByGoal', 'PartOf', 'ReceivesAction', 'UsedFor',
       'LocatedNear'], dtype=object)

In [5]:
relaList = result['relation'].values.tolist()

In [6]:
import numpy as np
dict(zip(*np.unique(relaList, return_counts=True)))

{'AtLocation': 1969298,
 'CapableOf': 6145829,
 'Causes': 50439,
 'CausesDesire': 249755,
 'CreatedBy': 52957,
 'DefinedAs': 36,
 'Desires': 10668,
 'HasA': 1562961,
 'HasFirstSubevent': 2962,
 'HasLastSubevent': 14048,
 'HasPrerequisite': 43131,
 'HasProperty': 2127824,
 'HasSubevent': 18904,
 'InstanceOf': 777688,
 'LocatedNear': 19,
 'MadeOf': 114111,
 'MotivatedByGoal': 15322,
 'PartOf': 357486,
 'ReceivesAction': 1492915,
 'UsedFor': 3475254}

In [7]:
dic = {'AtLocation': 1969298,
 'CapableOf': 6145829,
 'Causes': 50439,
 'CausesDesire': 249755,
 'CreatedBy': 52957,
 'DefinedAs': 36,
 'Desires': 10668,
 'HasA': 1562961,
 'HasFirstSubevent': 2962,
 'HasLastSubevent': 14048,
 'HasPrerequisite': 43131,
 'HasProperty': 2127824,
 'HasSubevent': 18904,
 'InstanceOf': 777688,
 'LocatedNear': 19,
 'MadeOf': 114111,
 'MotivatedByGoal': 15322,
 'PartOf': 357486,
 'ReceivesAction': 1492915,
 'UsedFor': 3475254}